<a href="https://colab.research.google.com/github/hs4704/Ev-Analysis/blob/main/EV-Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DSE 6000 - Final Project
## Tracking global data on electric vehicles

## Imports

In [ ]:
!pip install dash

In [ ]:
import dash
from dash import dcc, html
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
from statsmodels.datasets import get_rdataset
import sklearn.model_selection as skm
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from dash.dependencies import Input, Output
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error

## Upload Data

In [ ]:
evcars = pd.read_csv('/content/IEA-EV-dataEV salesHistoricalCars.csv')

In [ ]:
projected_evcars = pd.read_csv('/content/IEA-EV-dataEV salesProjection-STEPSCars.csv')

## View Data and Clean

In [ ]:
print(evcars.head())

      region    category       parameter  mode powertrain  year      unit  \
0  Australia  Historical        EV sales  Cars        BEV  2011  Vehicles   
1  Australia  Historical  EV stock share  Cars         EV  2011   percent   
2  Australia  Historical  EV sales share  Cars         EV  2011   percent   
3  Australia  Historical        EV stock  Cars        BEV  2011  Vehicles   
4  Australia  Historical        EV stock  Cars        BEV  2012  Vehicles   

       value  
0   49.00000  
1    0.00039  
2    0.00650  
3   49.00000  
4  220.00000  


In [ ]:
print(projected_evcars.head())

  region          category                      parameter  mode powertrain  \
0  China  Projection-STEPS             Electricity demand  Cars         EV   
1  China  Projection-STEPS           Oil displacement Mbd  Cars         EV   
2  China  Projection-STEPS  Oil displacement, million lge  Cars         EV   
3  China  Projection-STEPS                       EV sales  Cars        BEV   
4  China  Projection-STEPS                       EV stock  Cars        BEV   

   year                           unit        value  
0  2020                            GWh     9600.000  
1  2020         Milion barrels per day        0.051  
2  2020  Oil displacement, million lge     3000.000  
3  2020                       Vehicles   920000.000  
4  2020                       Vehicles  3500000.000  


There are no null values in the dataset

In [ ]:
evcars.isnull().values.any()

False

In [ ]:
evcars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3798 entries, 0 to 3797
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   region      3798 non-null   object 
 1   category    3798 non-null   object 
 2   parameter   3798 non-null   object 
 3   mode        3798 non-null   object 
 4   powertrain  3798 non-null   object 
 5   year        3798 non-null   int64  
 6   unit        3798 non-null   object 
 7   value       3798 non-null   float64
dtypes: float64(1), int64(1), object(6)
memory usage: 237.5+ KB


In [ ]:
projected_evcars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   region      451 non-null    object 
 1   category    451 non-null    object 
 2   parameter   451 non-null    object 
 3   mode        451 non-null    object 
 4   powertrain  451 non-null    object 
 5   year        451 non-null    int64  
 6   unit        451 non-null    object 
 7   value       451 non-null    float64
dtypes: float64(1), int64(1), object(6)
memory usage: 28.3+ KB


Review unique values columns (except for 'value' column')

In [ ]:
for column in evcars.columns[:-1]:
    print(f"Unique values in column '{column}':")
    print(evcars[column].unique())
    print()

Unique values in column 'region':
['Australia' 'Austria' 'Belgium' 'Brazil' 'Bulgaria' 'Canada' 'Chile'
 'China' 'Colombia' 'Costa Rica' 'Croatia' 'Cyprus' 'Czech Republic'
 'Denmark' 'Estonia' 'EU27' 'Europe' 'Finland' 'France' 'Germany' 'Greece'
 'Hungary' 'Iceland' 'India' 'Ireland' 'Israel' 'Italy' 'Japan' 'Korea'
 'Latvia' 'Lithuania' 'Luxembourg' 'Mexico' 'Netherlands' 'New Zealand'
 'Norway' 'Poland' 'Portugal' 'Rest of the world' 'Romania' 'Seychelles'
 'Slovakia' 'Slovenia' 'South Africa' 'Spain' 'Sweden' 'Switzerland'
 'Turkiye' 'United Arab Emirates' 'United Kingdom' 'USA' 'World']

Unique values in column 'category':
['Historical']

Unique values in column 'parameter':
['EV sales' 'EV stock share' 'EV sales share' 'EV stock'
 'Electricity demand' 'Oil displacement Mbd'
 'Oil displacement, million lge']

Unique values in column 'mode':
['Cars']

Unique values in column 'powertrain':
['BEV' 'EV' 'PHEV' 'FCEV']

Unique values in column 'year':
[2011 2012 2013 2014 2015 2016 20

In [ ]:
for column in projected_evcars.columns[:-1]:
    print(f"Unique values in column '{column}':")
    print(projected_evcars[column].unique())
    print()

Unique values in column 'region':
['China' 'Europe' 'India' 'Rest of the world' 'USA' 'World']

Unique values in column 'category':
['Projection-STEPS']

Unique values in column 'parameter':
['Electricity demand' 'Oil displacement Mbd'
 'Oil displacement, million lge' 'EV sales' 'EV stock' 'EV stock share'
 'EV sales share']

Unique values in column 'mode':
['Cars']

Unique values in column 'powertrain':
['EV' 'BEV' 'PHEV' 'FCEV']

Unique values in column 'year':
[2020 2021 2022 2023 2025 2030 2035]

Unique values in column 'unit':
['GWh' 'Milion barrels per day' 'Oil displacement, million lge' 'Vehicles'
 'percent']



Drop "category" and "mode" - no unique values and do not add value to the dataset

In [ ]:
evcars = evcars.drop(columns=['category', 'mode'])
print(evcars.head())

      region       parameter powertrain  year      unit      value
0  Australia        EV sales        BEV  2011  Vehicles   49.00000
1  Australia  EV stock share         EV  2011   percent    0.00039
2  Australia  EV sales share         EV  2011   percent    0.00650
3  Australia        EV stock        BEV  2011  Vehicles   49.00000
4  Australia        EV stock        BEV  2012  Vehicles  220.00000


In [ ]:
projected_evcars = projected_evcars.drop(columns=['category', 'mode'])
print(projected_evcars.head())

  region                      parameter powertrain  year  \
0  China             Electricity demand         EV  2020   
1  China           Oil displacement Mbd         EV  2020   
2  China  Oil displacement, million lge         EV  2020   
3  China                       EV sales        BEV  2020   
4  China                       EV stock        BEV  2020   

                            unit        value  
0                            GWh     9600.000  
1         Milion barrels per day        0.051  
2  Oil displacement, million lge     3000.000  
3                       Vehicles   920000.000  
4                       Vehicles  3500000.000  


**Cleaning Steps Summary**

After uploading and viewing the files, we ran functions to drop any null/na values. Luckily there weren't any!

We also determined that we could drop the 'category' and 'model' columns from our dataframes, because they were not providing any useful information.

When creating the visualizations below, some opted to create new columns or categories, such as when Jon made the other continental classifications for the 'region' column.

Summary Statistics (Year, Value)

In [ ]:
evcars.describe()

,year,value
count,3798.000000,3.798000e+03
mean,2017.369932,1.009542e+05
std,3.809226,8.184402e+05
min,2010.000000,1.500000e-05
25%,2014.000000,2.300000e+00
50%,2018.000000,1.900000e+02
75%,2021.000000,6.800000e+03
max,2023.000000,2.800000e+07


In [ ]:
projected_evcars.describe()

,year,value
count,451.000000,4.510000e+02
mean,2025.128603,4.632860e+06
std,5.044808,2.402404e+07
min,2020.000000,1.500000e-04
25%,2021.000000,1.900000e+01
50%,2023.000000,1.600000e+04
75%,2030.000000,9.400000e+05
max,2035.000000,3.900000e+08


Correlation

In [ ]:
evcars.select_dtypes(include='number').corr()

,year,value
year,1.000000,0.124555
value,0.124555,1.000000


In [ ]:
projected_evcars.select_dtypes(include='number').corr()

,year,value
year,1.000000,0.230629
value,0.230629,1.000000


## Visualizations

### Jon Visualizations

#### **(Replicated Viz) Heat Map of New Electric Carts Sold in 2023**

In [ ]:
#Filter data for 2023 and 'EV sales' parameter
filtered_df = evcars[(evcars['year'] == 2023) & (evcars['parameter'] == 'EV sales')]

#Group data by region and sum the values
summed_values_df = filtered_df.groupby('region')['value'].sum().reset_index()

#Create the choropleth map
jon1 = px.choropleth(summed_values_df,
                     locations="region",
                     locationmode="country names",  # Use full country names
                     color="value",
                     hover_name="region",
                     color_continuous_scale="peach",  # You can adjust the color scale if needed
                     title="Number of New Electric Cars Sold in 2023")

jon1.show()

**Analysis**

This first visualization shows a heat map of the number of electric cars sold in 2023. It shows the intense regionality of electric cars as a product, with the overwhelming majority of EVs being sold in China, Europe, North Americam, and Oceania.

Virtually no EVs were sold in Africa, central Asia, or the middle east.

#### **(Replicated Viz) Number of Cars Sold By Type - World**

In [ ]:
electric_cars_df = evcars[(evcars['region'] == 'World') & (evcars['parameter'] == 'EV sales')]
electric_cars_by_year = electric_cars_df.groupby('year')['value'].sum().reset_index()

# Filter for EV sales share data
ev_sales_share_df = evcars[(evcars['region'] == 'World') & (evcars['parameter'] == 'EV sales share')]

# Initialize an empty dictionary to store total sales by year
total_sales_by_year = {}

# Loop through each year to calculate total sales
for year in electric_cars_by_year['year']:
    # Get the EV sales share percentage for the given year
    ev_share = ev_sales_share_df[ev_sales_share_df['year'] == year]['value'].values[0] / 100

    # Get the electric car sales for the year
    electric_sales = electric_cars_by_year[electric_cars_by_year['year'] == year]['value'].values[0]

    # Calculate the total sales using the formula: Total Sales = Electric Sales / EV Sales Share
    total_sales = electric_sales / ev_share

    # Calculate the non-electric car sales
    non_electric_sales = total_sales - electric_sales

    # Store the values in the dictionary
    total_sales_by_year[year] = {
        'electric_sales': electric_sales,
        'non_electric_sales': non_electric_sales
    }

# Convert the dictionary into a DataFrame
stacked_sales_data = []
for year, sales in total_sales_by_year.items():
    stacked_sales_data.append({
        'year': year,
        'car_type': 'Electric',
        'value': sales['electric_sales']
    })
    stacked_sales_data.append({
        'year': year,
        'car_type': 'Non-Electric',
        'value': sales['non_electric_sales']
    })

# Convert to a DataFrame
stacked_sales_df = pd.DataFrame(stacked_sales_data)

# Create the stacked bar chart for the entire range of years (2010-2023)
jon2 = go.Figure()

# Add non-electric car sales (this will be the bottom part of the stack)
jon2.add_trace(go.Bar(
    x=stacked_sales_df[stacked_sales_df['car_type'] == 'Non-Electric']['year'],
    y=stacked_sales_df[stacked_sales_df['car_type'] == 'Non-Electric']['value'],
    name='Non-Electric Cars',
    marker_color='#CC5500'
))

# Add electric car sales (this will be the top part of the stack)
jon2.add_trace(go.Bar(
    x=stacked_sales_df[stacked_sales_df['car_type'] == 'Electric']['year'],
    y=stacked_sales_df[stacked_sales_df['car_type'] == 'Electric']['value'],
    name='Electric Cars',
    marker_color='#ADD8E6'
))

# Update layout for the stacked bar chart
jon2.update_layout(
    barmode='stack',
    title="Number of New Cars Sold (Electric vs Non-Electric) by Year (2010-2023)",
    xaxis_title="Year",
    yaxis_title="Number of Cars Sold",
    showlegend=True
)

jon2.show()

**Analysis**

This visualization shows the proportion of ALL car sales that are EVs, between 2010 and 2023. The big takeaway here is that, despite the explosive growth of EV car sales in recent years, it is still very much a small part of the overall global auto market.

#### **(Replicated Viz) - Share of cars currently in use that are electric, 2010 to 2023**

In [ ]:
# List of regions to include
regions_to_include = ['Norway', 'United Kingdom', 'World', 'China', 'Sweden']

# Filter the dataset for the selected regions and the 'EV stock share' parameter
ev_stock_share_filtered = evcars[(evcars['region'].isin(regions_to_include)) & (evcars['parameter'] == 'EV stock share')]

# Filter for the years 2010 to 2023
ev_stock_share_filtered = ev_stock_share_filtered[(ev_stock_share_filtered['year'] >= 2010) & (ev_stock_share_filtered['year'] <= 2023)]

# Divide the 'value' column by 100 to get the value in decimal format
ev_stock_share_filtered['value'] = ev_stock_share_filtered['value'] / 100

# Create the line plot
jon3 = px.line(ev_stock_share_filtered,
              x='year',
              y='value',  # Plot the EV stock share values (now in decimal format)
              color='region',  # Each line represents a different region
              title="EV Stock Share Over Time (2010-2023)",
              labels={'value': 'EV Stock Share (%)', 'year': 'Year'},
              line_shape='linear'  # Use linear lines
)

# Adjust y-axis to show percentage (with 2 decimal places)
jon3.update_yaxes(tickformat='.2%')  # This limits the decimal places to 2
jon3.show()

**Analysis**

In this dataset, "stock" represents the number of cars currently on the road and in use. The plot above shows the share of car stock in each country that is electric.

This plot nicely shows the major uptick in EV use over the last few years, with Norway being big early adopters. In 2023 a staggering 1/3 of cars on the road in Norway are electric, with neighbor Sweden in a distant second place with roughly 1/10.

Globally only about 3% of cars in active use are currently EVs, but that is a big increase from even 5 years ago.

#### **(Replicated Viz) Electric car stocks 2010 to 2023**

In [ ]:
# List of regions to include
regions_to_include = ['World', 'China', 'EU27', 'USA']

# Filter the dataset for the selected regions and the 'EV stock' parameter
ev_stock_filtered = evcars[(evcars['region'].isin(regions_to_include)) & (evcars['parameter'] == 'EV stock')]

# Filter for the years 2010 to 2023
ev_stock_filtered = ev_stock_filtered[(ev_stock_filtered['year'] >= 2010) & (ev_stock_filtered['year'] <= 2023)]

# Group the data by 'region' and 'year', then sum the values in the 'value' column
ev_stock_aggregated = ev_stock_filtered.groupby(['year', 'region'])['value'].sum().reset_index()

# Create the line plot for total EV stock
jon4 = px.line(ev_stock_aggregated,
              x='year',
              y='value',  # Plot the total EV stock values (aggregated)
              color='region',  # Each line represents a different region
              title="Total Number of Electric Car Stocks Over Time (2010-2023)",
              labels={'value': 'Total EV Stock', 'year': 'Year'},
              line_shape='linear'  # Use linear lines
)
jon4.show()

**Analysis**

This plot looks similar, but instead of showing a percentage it is showing the total count of EVs on the road in the countries/regions.

China is a big country by just about any metric, and they are currently dominating this plot with over 40 Million EVs. This is more than the USA and the EU combined.

#### **(New Viz) EV Sales for BRICS Nations Over Time**

In [ ]:
# Assuming 'evcars' DataFrame is already loaded
brics_nations = ['Brazil', 'China', 'India', 'South Africa', 'United Arab Emirates']
brics_df = evcars[(evcars['region'].isin(brics_nations)) & (evcars['parameter'] == 'EV sales share')]

# Group by region (country) and year, summing the EV sales
ev_sales_by_brics = brics_df.groupby(['region', 'year'])['value'].sum().reset_index()

# Create a Bubble Chart using Plotly Express
jon5 = px.scatter(ev_sales_by_brics,
                 x='year',
                 y='region',
                 size='value',  # Size of the bubbles proportional to EV sales
                 color='region',  # Color by country
                 hover_name='region',  # Show country name when hovering over the bubble
                 size_max=60,  # Maximum bubble size
                 title="EV Sales Share (% of Car Sales) by BRICS Nations (2010-2023)",
                 labels={"value": "EV Sales Share (% of Car Sales)", "year": "Year", "region": "Country"})
jon5.show()


**Analysis**

This fun bubble chart shows a comparison of the BRICS nations that were included in this dataset. BRICS officially stands for Brazil, Russia, India, China, and South Africa, but also includes Iran, Egypt, Ethiopia, and the United Arab Emirates. It is an informal group of emerging market economies that are often discussed in business and politics.

This plot shows the % of car sales in the BRICS nations, with the size of the circle being a proportion compared to the other countries in the dataset.

Again, China dominates here, with nearly 40% of new car sales in 2023, but the UAE is accelerating quickly, and the other countries showing big jumps since 2020.

#### **(New Viz) - EV Sales By Continent**

In [ ]:

# Example of a region filter and new grouped values.
regions_to_include = ['Australia', 'New Zealand', 'Brazil', 'Chile', 'Canada', 'Costa Rica', 'Mexico',
                      'USA', 'China', 'India', 'Israel', 'Japan', 'Korea', 'Turkiye', 'South Africa', 'Europe']

# Filter the dataset to include only the specified regions
ev_sales_demand_filtered = evcars[evcars['region'].isin(regions_to_include)]

# Further filter the dataset to include only 'EV sales'
ev_sales_demand_filtered = ev_sales_demand_filtered[ev_sales_demand_filtered['parameter'] == 'EV sales']

# Create a new 'continent' column based on 'region'
ev_sales_demand_filtered['continent'] = ev_sales_demand_filtered['region'].replace({
    'Australia': 'Oceania',  # Group Australia under Oceania
    'New Zealand': 'Oceania',  # Group New Zealand under Oceania
    'Brazil': 'South America',  # Group Brazil under South America
    'Chile': 'South America',  # Group Chile under South America
    'Canada': 'North America',  # Group Canada under North America
    'Costa Rica': 'North America',  # Group Costa Rica under North America
    'Mexico': 'North America',  # Group Mexico under North America
    'USA': 'North America',  # Group USA under North America
    'China': 'Asia',  # Group China under Asia
    'India': 'Asia',  # Group India under Asia
    'Israel': 'Asia',  # Group Israel under Asia
    'Japan': 'Asia',  # Group Japan under Asia
    'Korea': 'Asia',  # Group Korea under Asia
    'Turkiye': 'Asia',  # Group Turkiye under Asia
    'South Africa': 'Africa',  # Group South Africa under Africa
    'Europe': 'Europe'  # Europe remains as-is
})

# Aggregating EV Sales by Year and Continent
ev_sales_agg = ev_sales_demand_filtered.groupby(['year', 'continent'])['value'].sum().reset_index()

# Create separate bar plots for each continent, faceted by continent
jon6 = px.bar(ev_sales_agg,
             x='year',
             y='value',
             color='continent',  # Color by continent
             facet_col='continent',  # Separate plots by continent
             title="EV Sales by Continent",
             labels={'value': 'EV Sales', 'year': 'Year'},
             height=600  # Adjust height to make sure plots are properly visible
)
jon6.show()

**Analysis**

This plot splots up the total EV sales into continental boxes, and it backs up the information we saw in the global heatmap. Most EV car sales are happening in Asia, Europe, and North America, with the other continents falling extremely far behind. These may be markets to invest in.

### Mia Visualizations

#### Additional Exploration

In [ ]:
ev_sales_share_df = evcars[evcars['parameter'] == 'EV sales']
powertrain_counts = ev_sales_share_df['powertrain'].value_counts()
print("Count of each powertrain type (for 'EV sales share'):")
print(powertrain_counts)

Count of each powertrain type (for 'EV sales share'):
powertrain
BEV     604
PHEV    512
FCEV    226
Name: count, dtype: int64


In [ ]:
ev_sales_share_df = evcars[evcars['parameter'] == 'EV sales']
ev_sales_share_df = ev_sales_share_df[(ev_sales_share_df['year'].between(2010, 2023)) &
                                      (ev_sales_share_df['region'] == 'World')]
powertrain_sales_by_year = ev_sales_share_df.groupby(['year', 'powertrain'])['value'].sum().unstack(fill_value=0)
print("Sum of EV sales by powertrain type (by year, from 2010 to 2023, for 'World' region):")
print(powertrain_sales_by_year)

Sum of EV sales by powertrain type (by year, from 2010 to 2023, for 'World' region):
powertrain        BEV     FCEV       PHEV
year                                     
2010           7000.0     29.0      450.0
2011          40000.0     38.0     9000.0
2012          59000.0     30.0    61000.0
2013         110000.0     46.0    91000.0
2014         200000.0     94.0   130000.0
2015         330000.0    710.0   220000.0
2016         470000.0   2400.0   290000.0
2017         760000.0   3500.0   420000.0
2018        1400000.0   4500.0   660000.0
2019        1500000.0   7500.0   580000.0
2020        2000000.0   7400.0   980000.0
2021        4700000.0  16000.0  1900000.0
2022        7300000.0  15000.0  2900000.0
2023        9500000.0   8900.0  4300000.0


#### **(Replicated Viz) - Share of new cars sold that are battery-electric and plug-in hybrid, 2010 to 2023 (Bar)**

In [ ]:
#EV sales Share (percent of ev cars sold)
filter_data = evcars[(evcars['parameter'] == 'EV sales') |
                               (evcars['parameter']== 'EV sales share')]
filter_data = filter_data[filter_data['powertrain'] != 'FCEV']
filter_data = filter_data[filter_data['region'].isin(['World', 'Norway', 'United Kingdom', 'EU27', 'China', 'USA'])]
filter_data = filter_data[(filter_data['year'] >= 2011) & (filter_data['year'] <= 2023)]

#Split the data into EV sales and EV sales share
ev_sales_data = filter_data[filter_data['parameter'] == 'EV sales'].reset_index(drop=True)
ev_sales_share_data = filter_data[filter_data['parameter'] == 'EV sales share'].reset_index(drop=True)

#Merge
merged_data = pd.merge(ev_sales_data, ev_sales_share_data, on=['region', 'year'], suffixes=('_sales', '_share'))

#Calculate total ev sales per year, per region by adding value_sales per year per region
total_ev_sales = merged_data.groupby(['year', 'region'])['value_sales'].sum().reset_index()

#Merge total_ev_sales back with merged_data
merged_data = pd.merge(merged_data, total_ev_sales, on=['year', 'region'], suffixes=('', '_total'))

#Calculate BEV and PHEV shares per region per year by: value_sales/value_sales_total * value_share for each powertrain_sales
merged_data['Sales Share (%)'] = ((merged_data['value_sales'] / merged_data['value_sales_total']) * merged_data['value_share']).round()

#Merged_data[(merged_data['year']==2023) & (merged_data['region']=='World')]

bev_data = merged_data[merged_data['powertrain_sales'] == 'BEV']
phev_data = merged_data[merged_data['powertrain_sales'] == 'PHEV']

mia1 = make_subplots(
    rows=2, cols=3,
    subplot_titles=['World', 'Norway', 'United Kingdom', 'EU27', 'China', 'USA'],
    shared_yaxes=True,
    vertical_spacing=0.1
)

powertrain_colors = {'BEV': 'red', 'PHEV': 'blue'}

regions_of_interest = ['World', 'Norway', 'United Kingdom', 'EU27', 'China', 'USA']
for i, region in enumerate(regions_of_interest):
    row = i // 3 + 1
    col = i % 3 + 1

    region_bev_data = bev_data[bev_data['region'] == region]
    region_phev_data = phev_data[phev_data['region'] == region]

    mia1.add_trace(go.Bar(
        x=region_bev_data['year'],
        y=region_bev_data['Sales Share (%)'],
        name='BEV',
        marker=dict(color=powertrain_colors['BEV']),
        showlegend=True if i == 0 else False
    ), row=row, col=col)

    mia1.add_trace(go.Bar(
        x=region_phev_data['year'],
        y=region_phev_data['Sales Share (%)'],
        name='PHEV',
        marker=dict(color=powertrain_colors['PHEV']),
        showlegend=True if i == 0 else False
    ), row=row, col=col)

mia1.update_layout(
    title='Share of new cars sold that are BEV and PHEV, 2011 to 2023',
    barmode='stack',
    height=800,
    showlegend=True,
    legend=dict(
        title='Powertrain',
        x=1.05,
        y=1,
        xanchor='left',
        yanchor='top',
        orientation='v'
    ),
    xaxis_title='Year',
    yaxis_title='Sales Share (%)',
)

mia1.show()

**Analysis**

This plot shows the growth of EVs in a handful of regions, but most interestingly shows the split between full-battery EVs (BEV) and plug-in hybrids (PHEV).

The full-battery EVs are more popular in all regions when compared to the hybrids. In the regions shown, the full-battery EVs are twice as popular, making up 12% of sales globally in 2023, vs 6% for the hybrids.

One can predict that the trend for both types of EV to continue this upward trend.

#### **(Replicated Viz) - Share of new electric cars that are fully battery-electric, 2012 to 2023 (Line)**

In [ ]:
#Filter data to 'EV sales', specific regions and years 2012-2023
ev_sales_share_df = evcars[evcars['parameter'] == 'EV sales']

regions_of_interest = ['World', 'Norway', 'United Kingdom', 'China','Sweden']
ev_sales_share_df = ev_sales_share_df[(ev_sales_share_df['region'].isin(regions_of_interest)) &
                                      (ev_sales_share_df['year'].between(2012, 2023))]

ev_sales_aggregated = ev_sales_share_df.groupby(['region', 'year', 'powertrain'], as_index=False)['value'].sum()

#Total sales by region and year (sum of BEV + PHEV + FCEV sales)
total_sales = ev_sales_aggregated.groupby(['region', 'year'])['value'].sum().reset_index()
total_sales = total_sales.rename(columns={'value': 'Total'})

ev_sales_aggregated = pd.merge(ev_sales_aggregated, total_sales, on=['region', 'year'])

#Calculate % of BEV (battery-electric) from total EV sales
ev_sales_aggregated['Percentage'] = (ev_sales_aggregated['value'] / ev_sales_aggregated['Total']) * 100

ev_sales_bv = ev_sales_aggregated[ev_sales_aggregated['powertrain'] == 'BEV']

region_colors = {
    'World': 'purple',
    'Norway': 'red',
    'United Kingdom': 'green',
    'China': 'blue',
    'Sweden': 'gold'
}

mia2 = go.Figure()

for region in regions_of_interest:
    region_data = ev_sales_bv[ev_sales_bv['region'] == region]

    #Add the trace
    mia2.add_trace(go.Scatter(
        x=region_data['year'],
        y=region_data['Percentage'],
        mode='lines',
        name=region,
        line=dict(color=region_colors[region])
    ))

mia2.update_layout(
    title='Share of new electric cars that are fully battery-electric, 2012 to 2023',
    xaxis_title='Year',
    yaxis_title='Percentage of BEV Sales (%)',
    height=600,
    showlegend=True
)

mia2.show()

**Analysis**


*   European countries such as Norway, Sweden and the United Kingdom saw a noticeable declines or stagnation in the share of new, fully battery-electric cars sold between 2015 - 2017. Afterwards, the share of these vehicles sold began to increase.
*   In comparison, China observed a relatively consistent and strong share in the sales of fully battery-electric cars after 2014. However, the country entered an annual decline beginning in 2021.



#### **(New Viz) - EV Stock vs Sales Ratio by Region**

In [ ]:
evcars_filtered = evcars[(evcars['year'] >= 2010) & (evcars['year'] <= 2023)]
df_sales = evcars_filtered[evcars_filtered['parameter'] == 'EV sales']
df_stock = evcars_filtered[evcars_filtered['parameter'] == 'EV stock']

df_sales_grouped = df_sales.groupby(['region', 'year'])['value'].sum().reset_index()
df_stock_grouped = df_stock.groupby(['region', 'year'])['value'].sum().reset_index()

df_merged = pd.merge(df_sales_grouped, df_stock_grouped, on=['region', 'year'], suffixes=('_sales', '_stock'))

df_merged['EV Stock vs Sales Ratio'] = df_merged['value_stock'] / df_merged['value_sales']

latest_year = 2023 if 2023 in df_merged['year'].values else df_merged['year'].max()
df_latest_year = df_merged[df_merged['year'] == latest_year]

choropleth_data = df_latest_year[['region', 'EV Stock vs Sales Ratio']]

mia3 = px.choropleth(
    choropleth_data,
    locations="region",
    color="EV Stock vs Sales Ratio",
    color_continuous_scale='RdYlGn',
    title=f"EV Stock vs. EV Sales Ratio by Region ({latest_year})",
    locationmode="country names",
    labels={"EV Stock vs Sales Ratio": "Stock/Sales Ratio"},
    template="plotly"
)

mia3.show()

**Analysis**

The ratio of EV stock to EV sales for the most recent year (2023) indicates how many EVs a country has on the road in comparison to how many they are selling.

For countries like Norway, this shows that they have a lot of EVS on the road already and the market may already becoming saturated for EVs. In comparison, a country such as Brazil, India, or Australia that has a lower ratio indicates that the country is selling a lot of EVs, but does not have many EVs already on the road as of the most recent year. These countries could exemplify a desirable or up-and-coming market for EV sales.

#### **(New Viz) Proportional EV Sales in USA (2011-2023) (Treemap)**

In [ ]:
region_of_interest = ["USA"]

df_filtered = evcars[evcars['region'].isin(region_of_interest)]

mia4 = px.treemap(df_filtered,
                 path=["region", "year"],
                 values="value",
                 title="Proportional EV Sales in USA (2011-2023)",
                 labels={"value": "Number of Vehicles", "region": "Region"},
                 color="value",
                 color_continuous_scale="Blues"
                )

mia4.update_layout(
    title="Proportional EV Sales in USA (2011-2023)",
    height=600,
    width=900
)
mia4.show()

**Analysis**

*   Annually, EV sales have grown significantly in the US. Proportionally, sales in the 2020's have been significantly compared to the 2010's.



### Dani Visualizations

#### **(Replicated Viz) EV Sales Share (2010-2023) by Region**

In [ ]:
evcars_historical= evcars
evcars.head()

,region,parameter,powertrain,year,unit,value
0,Australia,EV sales,BEV,2011,Vehicles,49.00000
1,Australia,EV stock share,EV,2011,percent,0.00039
2,Australia,EV sales share,EV,2011,percent,0.00650
3,Australia,EV stock,BEV,2011,Vehicles,49.00000
4,Australia,EV stock,BEV,2012,Vehicles,220.00000


In [ ]:
# Filter the data for 'EV sales share' category
ev_sales_share_df = evcars_historical[evcars_historical['parameter'] == 'EV sales share']
regions_of_interest = ['World', 'Norway', 'United Kingdom', 'EU27', 'China', 'USA']
ev_sales_share_df = ev_sales_share_df[ev_sales_share_df['region'].isin(regions_of_interest)]
# Aggregate data by summing the EV sales share for each region and year
ev_sales_share_aggregated = ev_sales_share_df.groupby(['region', 'year'], as_index=False)['value'].sum()

y_axis_range = [0, 100]

# Create a subplot layout with 2 rows and 3 columns
Dani1 = make_subplots(
    rows=2, cols=3,
    subplot_titles=regions_of_interest,  # Set titles for each subplot
    shared_yaxes=True,  # Share y-axis across all subplots
)

# Loop through each region to create a line plot and add it to the subplot grid
for i, region in enumerate(regions_of_interest):
    row = i // 3 + 1  # Calculate row number (1 or 2)
    col = i % 3 + 1   # Calculate column number (1, 2, or 3)

    region_data = ev_sales_share_aggregated[ev_sales_share_aggregated['region'] == region]

    # Create the line plot for the region
    trace = go.Scatter(
        x=region_data['year'],
        y=region_data['value'],
        mode='lines',
        name=region,
        hovertext=region_data.apply(lambda row: f"Year: {row['year']}<br>Share of new cars that are electric: {row['value']: .0f}%", axis=1)  # Custom hovertext
    )

    # Add the trace to the subplot grid
    Dani1.add_trace(trace, row=row, col=col)

# Update the layout for the whole Figure
Dani1.update_layout(
    title='EV Sales Share (2010-2023) by Region',
    height=800,  # Set the height of the plot
    showlegend=False,  # Hide legend as we have titles for each subplot
)

# Update the y-axis range and manually add '%' to the ticks
Dani1.update_yaxes(range=y_axis_range, tickvals=[i for i in range(0, 101, 20)], ticktext=[f'{i}%' for i in range(0, 101, 20)])
Dani1.show()


**Analysis**

This plot shows the increase in EV sales around the world, measured as a percentage of all car sales.

Norway were the earliest adopters shown, with 1/3 of new cars sold being electric as early as 2016.

Interestingly, EV sales seem to be leveling off in the EU, as well as in the UK. Each have only shown a ~1% growth in the last year after several years of 4-5% growth.

#### **(Replicated Viz) EV Sales Share by Region in 2023**

In [ ]:
# Filter the data for 'EV sales share' category and the year 2023
ev_sales_share_2023 = evcars_historical[(evcars_historical['parameter'] == 'EV sales share') & (evcars_historical['year'] == 2023)]

# Further filter for specific regions
regions_of_interest = ['World', 'Norway', 'United Kingdom', 'EU27', 'China', 'USA','Sweden','Germany','India','South Africa']
ev_sales_share_2023 = ev_sales_share_2023[ev_sales_share_2023['region'].isin(regions_of_interest)]

ev_sales_share_2023_aggregated = ev_sales_share_2023.groupby(['region'], as_index=False)['value'].sum()

ev_sales_share_2023_aggregated = ev_sales_share_2023_aggregated.sort_values(by='value', ascending=True)

# Create a horizontal bar graph
Dani2 = px.bar(ev_sales_share_2023_aggregated,
             x='value',
             y='region',
             orientation='h',
             title='EV Sales Share by Region in 2023',
             text='value')
Dani2.update_traces( text= ev_sales_share_2023_aggregated['value'].apply(lambda x:"<1%" if x< 1 else f"{x: .0f}%"),
                  textposition='outside')

# Show the plot
Dani2.show()

**Analysis**

This plot shows the percentage of new cars sold that are electric for a number of countries, as well as the world as a whole, in 2023.

Scandanavia is leading the way, with almost all car sales in Norway being electric, and 2/3 in Sweden being electric.

#### **(New Viz) EV Stock Share vs. Oil Displacement (Scatter plot of regions)**

In [ ]:
ev_stock_share= evcars_historical[evcars_historical['parameter'] == 'EV stock share']
oil_displacement= evcars_historical[evcars_historical['parameter'] == 'Oil displacement Mbd']

ev_stock_share.head()

,region,parameter,powertrain,year,unit,value
1,Australia,EV stock share,EV,2011,percent,0.00039
8,Australia,EV stock share,EV,2012,percent,0.00240
11,Australia,EV stock share,EV,2013,percent,0.00460
20,Australia,EV stock share,EV,2014,percent,0.01400
23,Australia,EV stock share,EV,2015,percent,0.02700


In [ ]:
merge_data= pd.merge(ev_stock_share, oil_displacement, on=['region','year'], suffixes=('_ev', '_oil'))
merge_data.head()

,region,parameter_ev,powertrain_ev,year,unit_ev,value_ev,parameter_oil,powertrain_oil,unit_oil,value_oil
0,China,EV stock share,EV,2010,percent,0.0031,Oil displacement Mbd,EV,Milion barrels per day,0.000029
1,China,EV stock share,EV,2011,percent,0.0092,Oil displacement Mbd,EV,Milion barrels per day,0.000110
2,China,EV stock share,EV,2012,percent,0.0190,Oil displacement Mbd,EV,Milion barrels per day,0.000270
3,China,EV stock share,EV,2013,percent,0.0310,Oil displacement Mbd,EV,Milion barrels per day,0.000550
4,China,EV stock share,EV,2014,percent,0.0700,Oil displacement Mbd,EV,Milion barrels per day,0.001300


In [ ]:
Dani3= px.scatter(merge_data, x='value_ev', y='value_oil',
                color='region',
                size='value_oil',
                title='EV Stock Share vs Oil Displacement(Mbd)',
                labels={'value_ev':'Ev Stock Share(%)', 'value_oil':'Oil displacement (Mbd)'}
                )

Dani3.show()

**Analysis**

This graph shows how as EV stock Share increases so does oil displacemenmt meaning less consumption of gasoline and diesel fuels. You can see how the world has a pretty steep slope with oil consumption decreasing fast worldwide, but in countries like China, even with more EV tock share, they still use a lot of oil and fuel.

#### **(New Viz) Line Graph, Electricity Demand and Oil Displacement Over Time**

In [ ]:
#create new dataframe with parameters electricity demand and oil displacement
evcars_historical = pd.read_csv('/content/IEA-EV-dataEV salesHistoricalCars.csv')
electricity = evcars_historical[evcars_historical['parameter'] == 'Electricity demand']
oil_displacement = evcars_historical[evcars_historical['parameter'] == 'Oil displacement Mbd']
#merge the two dataframes by region and year
merged_data = pd.merge(electricity, oil_displacement, on=['region', 'year'])
merged_data.head()

,region,category_x,parameter_x,mode_x,powertrain_x,year,unit_x,value_x,category_y,parameter_y,mode_y,powertrain_y,unit_y,value_y
0,China,Historical,Electricity demand,Cars,EV,2010,GWh,46.0,Historical,Oil displacement Mbd,Cars,EV,Milion barrels per day,0.000029
1,China,Historical,Electricity demand,Cars,EV,2011,GWh,120.0,Historical,Oil displacement Mbd,Cars,EV,Milion barrels per day,0.000110
2,China,Historical,Electricity demand,Cars,EV,2012,GWh,230.0,Historical,Oil displacement Mbd,Cars,EV,Milion barrels per day,0.000270
3,China,Historical,Electricity demand,Cars,EV,2013,GWh,470.0,Historical,Oil displacement Mbd,Cars,EV,Milion barrels per day,0.000550
4,China,Historical,Electricity demand,Cars,EV,2014,GWh,660.0,Historical,Oil displacement Mbd,Cars,EV,Milion barrels per day,0.001300


In [ ]:
import plotly.graph_objects as go

aggregated_data = merged_data.groupby('year')[['value_x', 'value_y']].sum().reset_index()
aggregated_data.rename(columns={'value_x': 'Electricity Demand (GWh)', 'value_y': 'Oil Displacement (Mbd)'}, inplace=True)

Dani4 = go.Figure()

#Electricity Demand trace
Dani4.add_trace(
    go.Scatter(
        x=aggregated_data['year'],
        y=aggregated_data['Electricity Demand (GWh)'],
        name="Electricity Demand (GWh)",
        mode="lines+markers",
        line=dict(color='blue')
    )
)

# Oil Displacement trace
Dani4.add_trace(
    go.Scatter(
        x=aggregated_data['year'],
        y=aggregated_data['Oil Displacement (Mbd)'],
        name="Oil Displacement (Mbd)",
        mode="lines+markers",
        line=dict(color='orange'),
        yaxis="y2" ,
        marker=dict(symbol='triangle-up', size=10) # Changed symbol to 'triangle-up'
    )
)

# layout for dual axes
Dani4.update_layout(
    title="Electricity Demand and Oil Displacement Over Time",
    xaxis_title="Year",
    yaxis=dict(
        title="Electricity Demand (GWh)",
        titlefont=dict(color='blue'),
        tickfont=dict(color='blue'),
    ),
    yaxis2=dict(
        title="Oil Displacement (Mbd)",
        titlefont=dict(color='orange'),
        tickfont=dict(color='orange'),
        overlaying="y",
        side="right",
    ),
    legend=dict(
        x=1.2, y=1.0, xanchor="center", orientation="v"
    ),
    template="plotly"
)

Dani4.show()

**Analysis**

This graph shows how Electricity Demand is positively correlated to Oil Displacement. This means as Electricity demand goes up, the amount of Oil needed directly goes down as a result.

## Models

In [ ]:
world_filtered_df = evcars[(evcars['region'] == 'World') & (evcars['parameter'] == 'EV sales')]
x = world_filtered_df[['year','powertrain']]
x = pd.get_dummies(x, columns =['powertrain'], drop_first=False)
Y = world_filtered_df['value']
world_filtered_df.head()

,region,parameter,powertrain,year,unit,value
3647,World,EV sales,BEV,2010,Vehicles,7000.0
3649,World,EV sales,FCEV,2010,Vehicles,29.0
3651,World,EV sales,PHEV,2010,Vehicles,450.0
3658,World,EV sales,PHEV,2011,Vehicles,9000.0
3660,World,EV sales,FCEV,2011,Vehicles,38.0


In [ ]:
x_train, x_test, Y_train, Y_test = train_test_split(x, Y, test_size = 0.2, random_state = 42)

# Fit a linear regression model
linear_model = LinearRegression()
linear_model.fit(x_train, Y_train)
linear_pred = linear_model.predict(x_test)

# Fit a random forest model
rf_model = RandomForestRegressor(n_estimators = 100, random_state = 42)
rf_model.fit(x_train, Y_train)
rf_pred = rf_model.predict(x_test)

# Fit an XGBoost model
xgb_model = XGBRegressor(n_estimators = 100, random_state = 42)
xgb_model.fit(x_train, Y_train)
xgb_pred = xgb_model.predict(x_test)

# Calculate mean squared error for each model
linear_mse = mean_squared_error(Y_test, linear_pred)
rf_mse = mean_squared_error(Y_test, rf_pred)
xgb_mse = mean_squared_error(Y_test, xgb_pred)

# Print the results
print(f"Linear regression MSE: {linear_mse}")
print(f"Random forest MSE: {rf_mse}")
print(f"XGBoost MSE: {xgb_mse}")

Linear regression MSE: 1290042822390.317
Random forest MSE: 94697079822.11725
XGBoost MSE: 154814063742.15884


In [ ]:
projected_world_df = projected_evcars[(projected_evcars['region'] == 'World') & (projected_evcars['parameter'] == 'EV sales')]
projected_x = projected_world_df[['year','powertrain']]
projected_world_df.head()

,region,parameter,powertrain,year,unit,value
375,World,EV sales,FCEV,2020,Vehicles,7400.0
379,World,EV sales,PHEV,2020,Vehicles,980000.0
382,World,EV sales,BEV,2020,Vehicles,2000000.0
387,World,EV sales,BEV,2021,Vehicles,4700000.0
390,World,EV sales,PHEV,2021,Vehicles,1900000.0


In [ ]:
projected_x_dummies = pd.get_dummies(projected_x, columns =['powertrain'], drop_first=False)
projected_x_dummies.head()

,year,powertrain_BEV,powertrain_FCEV,powertrain_PHEV
375,2020,False,True,False
379,2020,False,False,True
382,2020,True,False,False
387,2021,True,False,False
390,2021,False,False,True


In [ ]:
projected_x.head()

,year,powertrain
375,2020,FCEV
379,2020,PHEV
382,2020,BEV
387,2021,BEV
390,2021,PHEV


In [ ]:
# Predict using each model
linear_projected_pred = linear_model.predict(projected_x_dummies)
rf_projected_pred = rf_model.predict(projected_x_dummies)
xgb_projected_pred = xgb_model.predict(projected_x_dummies)

# Add predictions to the projected dataset
projected_x_dummies['Linear_Predictions'] = linear_projected_pred
projected_x_dummies['RF_Predictions'] = rf_projected_pred
projected_x_dummies['XGB_Predictions'] = xgb_projected_pred

In [ ]:
actual_values = projected_world_df['value']
linear_mse = mean_squared_error(actual_values, linear_projected_pred)
rf_mse = mean_squared_error(actual_values, rf_projected_pred)
xgb_mse = mean_squared_error(actual_values, xgb_projected_pred)
years = projected_world_df['year']

In [ ]:
sonu1 = go.Figure()

# Add traces for each model
sonu1.add_trace(go.Scatter(
    x=projected_world_df['year'],
    y=actual_values,
    mode='lines+markers',
    name='Actual Values',
    line=dict(color='purple'),
    marker=dict(symbol='circle', size=8)
))

sonu1.add_trace(go.Scatter(
    x=projected_world_df['year'],
    y=linear_projected_pred,
    mode='lines+markers',
    name='Linear Predictions',
    line=dict(color='red'),
    visible=False  # Initially hidden
))

sonu1.add_trace(go.Scatter(
    x=projected_world_df['year'],
    y=rf_projected_pred,
    mode='lines+markers',
    name='Random Forest Predictions',
    line=dict(color='green'),
    visible=False  # Initially hidden
))

sonu1.add_trace(go.Scatter(
    x=projected_world_df['year'],
    y=xgb_projected_pred,
    mode='lines+markers',
    name='XGBoost Predictions',
    line=dict(color='blue'),
    visible=False  # Initially hidden
))

# Add buttons for each tab
sonu1.update_layout(
    updatemenus=[
        {
            'type': 'buttons',
            'buttons': [
                {
                    'label': 'All Models',
                    'method': 'update',
                    'args': [
                        {'visible': [True, True, True, True]},  # Show all traces
                        {'title': 'All Models: Actual vs Predicted'}
                    ]
                },
                {
                    'label': 'Linear Regression',
                    'method': 'update',
                    'args': [
                        {'visible': [True, True, False, False]},  # Show only actual and linear
                        {'title': 'Linear Regression: Actual vs Predicted'}
                    ]
                },
                {
                    'label': 'Random Forest',
                    'method': 'update',
                    'args': [
                        {'visible': [True, False, True, False]},  # Show only actual and random forest
                        {'title': 'Random Forest: Actual vs Predicted'}
                    ]
                },
                {
                    'label': 'XGBoost',
                    'method': 'update',
                    'args': [
                        {'visible': [True, False, False, True]},  # Show only actual and xgboost
                        {'title': 'XGBoost: Actual vs Predicted'}
                    ]
                }
            ]
        }
    ]
)

# Set layout details
sonu1.update_layout(
    title='Model Predictions vs Actual Sales',
    xaxis_title='Year',
    yaxis_title='Sales',
    showlegend=True
)

sonu1.show()

In [ ]:
actual_values = projected_world_df['value']
linear_mse = mean_squared_error(actual_values, linear_projected_pred)
rf_mse = mean_squared_error(actual_values, rf_projected_pred)
xgb_mse = mean_squared_error(actual_values, xgb_projected_pred)

print(f"Linear regression MSE: {linear_mse}")
print(f"Random forest MSE: {rf_mse}")
print(f"XGBoost MSE: {xgb_mse}")

Linear regression MSE: 133112969261009.23
Random forest MSE: 120291203566502.0
XGBoost MSE: 111248836667036.53


In [ ]:
import plotly.graph_objects as go

sonu2 = go.Figure()

sonu2.add_trace(go.Bar(name='Linear Regression', x=projected_world_df['year'], y=projected_x_dummies['Linear_Predictions']))
sonu2.add_trace(go.Bar(name='Random Forest', x=projected_world_df['year'], y=projected_x_dummies['RF_Predictions']))
sonu2.add_trace(go.Bar(name='XGBoost', x=projected_world_df['year'], y=projected_x_dummies['XGB_Predictions']))
sonu2.add_trace(go.Bar(name='Actual Values', x=projected_world_df['year'], y=projected_world_df['value']))

# Customize layout
sonu2.update_layout(
    title='Predicted vs. Actual Values',
    xaxis_title='Year',
    yaxis_title='Sales',
    barmode='group',  # Grouped bar graph
)

# Show the graph
sonu2.show()

In [ ]:
#Reverse dummy encoding to reconstruct the 'powertrain' column
powertrain_mapping = {
    "powertrain_FCEV": "FCEV",
    "powertrain_PHEV": "PHEV",
    "powertrain_BEV": "BEV"
}

# Create the 'powertrain' column
projected_x_dummies["powertrain"] = (
    projected_x_dummies[["powertrain_FCEV", "powertrain_PHEV", "powertrain_BEV"]]
    .idxmax(axis=1)
    .map(powertrain_mapping)
)

# Merge the datasets
merged_df = pd.merge(
    projected_world_df,
    projected_x_dummies,
    on=["year", "powertrain"],  # Merge on 'year' and 'powertrain'
    how="inner"  # Ensure only matching rows are included
)

# Extract relevant columns
merged_df = merged_df[
    ["year", "powertrain", "value", "Linear_Predictions", "RF_Predictions", "XGB_Predictions"]
]

# Melt the dataset for visualization
melted_df = merged_df.melt(
    id_vars=["year", "powertrain"],  # Keep 'year' and 'powertrain' as identifiers
    value_vars=["value", "Linear_Predictions", "RF_Predictions", "XGB_Predictions"],  # Melt these columns
    var_name="Type",  # Name for the variable column
    value_name="Values"  # Name for the value column
)


fig = px.bar(
    melted_df,
    x="year",
    y="Values",
    color="powertrain",  # Color by powertrain type
    barmode="group",  # Grouped bars
    facet_row="Type",  # Separate plots for actual values and predictions
    title="Actual vs Predicted EV Sales by Year and Powertrain",
    labels={"year": "Year", "Values": "Sales", "Type": "Data Type"}
)

fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Sales",
    bargap=0.15,  # Adjust spacing between bars
    legend_title="Powertrain"
)

fig.show()


In [ ]:
#Reverse dummy encoding to reconstruct the 'powertrain' column
powertrain_mapping = {
    "powertrain_FCEV": "FCEV",
    "powertrain_PHEV": "PHEV",
    "powertrain_BEV": "BEV"
}

# Create the 'powertrain' column
projected_x_dummies["powertrain"] = (
    projected_x_dummies[["powertrain_FCEV", "powertrain_PHEV", "powertrain_BEV"]]
    .idxmax(axis=1)
    .map(powertrain_mapping)
)

# Merge the datasets
merged_df = pd.merge(
    projected_world_df,
    projected_x_dummies,
    on=["year", "powertrain"],  # Merge on 'year' and 'powertrain'
    how="inner"  # Ensure only matching rows are included
)

# Extract relevant columns
merged_df = merged_df[
    ["year", "powertrain", "value", "Linear_Predictions", "RF_Predictions", "XGB_Predictions"]
]
melted_df = merged_df.melt(
    id_vars=["year", "powertrain"],  # Keep 'year' and 'powertrain' as identifiers
    value_vars=["value", "Linear_Predictions", "RF_Predictions", "XGB_Predictions"],  # Columns to melt
    var_name="Type",  # New column for type (Actual, Linear Regression, etc.)
    value_name="Values"  # New column for the melted values
)

# Create the grouped and stacked bar chart
sonu4 = px.bar(
    melted_df,
    x="year",  # Year on the x-axis
    y="Values",  # Sales values on the y-axis
    color="powertrain",  # Stack bars by powertrain type
    barmode="stack",  # Stack bars within the same group
    facet_col="Type",  # Separate bars by Type (Actual, Linear Regression, etc.)
    labels={"year": "Year", "Values": "Sales", "Type": "Prediction Type"},
    title="Actual vs Predicted EV Sales by Year and Powertrain"
)

sonu4.update_layout(
    xaxis_title="Year",
    yaxis_title="Sales",
    bargap=0.15,  # Adjust spacing between bars
    legend_title="Powertrain",
    title_x=0.5  # Center the title
)

sonu4.show()

**Analysis**

This visualization shows the predictions of the actual projections vs the machine learning models based on powertrain. Battery electric vehicles (BEV) have the most sales in all models

In [ ]:
# Predict using each model
linear_pred = linear_model.predict(x)
rf_pred = rf_model.predict(x)
xgb_pred = xgb_model.predict(x)

# Add predictions to the projected dataset
x['Linear_Predictions'] = linear_pred
x['RF_Predictions'] = rf_pred
x['XGB_Predictions'] = xgb_pred

In [ ]:
actual_original_values = world_filtered_df['value']
linear_mse1 = mean_squared_error(actual_original_values, linear_pred)
rf_mse1 = mean_squared_error(actual_original_values, rf_pred)
xgb_mse1 = mean_squared_error(actual_original_values, xgb_pred)

print(f"Linear regression MSE for past data: {linear_mse1}")
print(f"Random forest MSE for past data: {rf_mse1}")
print(f"XGBoost MSE for past data: {xgb_mse1}")

Linear regression MSE for past data: 2062005937098.6272
Random forest MSE for past data: 241132413641.44107
XGBoost MSE for past data: 33174442244.113457


In [ ]:
sonu3 = go.Figure()

sonu3.add_trace(go.Bar(name='Linear Regression', x=world_filtered_df['year'], y=x['Linear_Predictions']))
sonu3.add_trace(go.Bar(name='Random Forest', x=world_filtered_df['year'], y=x['RF_Predictions']))
sonu3.add_trace(go.Bar(name='XGBoost', x=world_filtered_df['year'], y=x['XGB_Predictions']))
sonu3.add_trace(go.Bar(name='Actual Values', x=world_filtered_df['year'], y=world_filtered_df['value']))

# Customize layout
sonu3.update_layout(
    title='Predicted vs. Actual Values',
    xaxis_title='Year',
    yaxis_title='Sales',
    barmode='group',  # Grouped bar graph
)

# Show the graph
sonu3.show()

In [ ]:
linear_MSE = 133112969261009.23
rf_MSE = 120291203566502.0
XGB_MSE = 111248836667036.53

linear_mse_sci = f"{linear_mse:.2e}"  # Convert to scientific notation
rf_mse_sci = f"{rf_mse:.2e}"
xgb_mse_sci = f"{xgb_mse:.2e}"

data = {
    "Model": ["Linear Regression", "Random Forest", "XGBoost"],
    "MSE (Scientific Notation)": [linear_mse_sci, rf_mse_sci, xgb_mse_sci]
}

comparison_df = pd.DataFrame(data)

# Sort the data to determine the model with the highest MSE
comparison_df['MSE Value'] = [linear_mse, rf_mse, xgb_mse]
comparison_df = comparison_df.sort_values(by="MSE Value", ascending=True)

# Display the table
print(comparison_df[['Model', 'MSE (Scientific Notation)']])

               Model MSE (Scientific Notation)
2            XGBoost                  1.11e+14
1      Random Forest                  1.20e+14
0  Linear Regression                  1.33e+14


In [ ]:
linear_MSE = 2062005937098.6272
rf_MSE = 241132413641.44107
XGB_MSE = 33174442244.113457

linear_mse_sci = f"{linear_MSE:.2e}"  # Convert to scientific notation
rf_mse_sci = f"{rf_MSE:.2e}"
xgb_mse_sci = f"{XGB_MSE:.2e}"

data = {
    "Model": ["Linear Regression", "Random Forest", "XGBoost"],
    "MSE (Scientific Notation)": [linear_mse_sci, rf_mse_sci, xgb_mse_sci]
}

comparison_df = pd.DataFrame(data)

# Sort the data to determine the model with the highest MSE
comparison_df['MSE Value'] = [linear_mse, rf_mse, xgb_mse]
comparison_df = comparison_df.sort_values(by="MSE Value", ascending=True)

# Display the table
print(comparison_df[['Model', 'MSE (Scientific Notation)']])

               Model MSE (Scientific Notation)
2            XGBoost                  3.32e+10
1      Random Forest                  2.41e+11
0  Linear Regression                  2.06e+12


## DASH APP


In [ ]:
from dash import Dash, dcc, html

app = Dash(__name__)

app.layout = html.Div([
    html.H1("EV Data Dash App", style={'text-align': 'center'}),
    # Combine both Tabs into a single dcc.Tabs
    dcc.Tabs([
        # Tab 1 - Replicated Visuals
        dcc.Tab(label="Replicated Visuals", children=[
            html.H2("Replicated Visuals", style={'text-align': 'center'}),
            html.Div([
                html.H2("Replicated Visual 1"),
                html.P("This plot shows the increase in EV sales around the world, measured as a percentage of all car sales."
                " Norway were the earliest adopters shown, with 1/3 of new cars sold being electric as early as 2016."
                " Interestingly, EV sales seem to be leveling off in the EU, as well as in the UK. Each have only shown a ~1% growth in the last year after several years of 4-5% growth."),
                dcc.Graph(id='ev-sales-share', figure=Dani1)
            ]),

            html.Div([
                html.H2("Replicated Visual 2"),
                html.P("This plot shows the percentage of new cars sold that are electric for a number of countries, as well as the world as a whole, in 2023."
                " Scandinavia is leading the way, with almost all car sales in Norway being electric, and 2/3 in Sweden being electric."),
                dcc.Graph(id='ev-sales-2023', figure=Dani2)
            ]),
            html.Div([
                html.H2("Replicated Visual 3"),
                html.P("This plot shows the growth of EVs in a handful of regions, but most interestingly shows the split between full-battery EVs (BEV) and plug-in hybrids (PHEV)."
                "The full-battery EVs are more popular in all regions when compared to the hybrids. In the regions shown, the full-battery EVs are twice as popular, making up 12% of sales globally in 2023, vs 6% for the hybrids."
                "One can predict that the trend for both types of EV to continue this upward trend."),
                dcc.Graph(id='ev-share-BEVPHEV', figure=mia1)
            ]),
            html.Div([
                html.H2("Replicated Visual 4"),
                html.P("European countries such as Norway, Sweden and the United Kingdom saw a noticeable declines or stagnation in the share of new, fully battery-electric cars sold between 2015 - 2017."
                "Afterwards, the share of these vehicles sold began to increase. In comparison, China observed a relatively consistent and strong share in the sales of fully battery-electric cars after 2014."
                "However, the country entered an annual decline beginning in 2021."),
                dcc.Graph(id='ev-share-BEV', figure=mia2)
            ]),
            html.Div([
                html.H2("Replicated Visual 5"),
                html.P("This first visualization shows a heat map of the number of electric cars sold in 2023."
                "It shows the intense regionality of electric cars as a product, with the overwhelming majority of EVs being sold in China, Europe, North Americam, and Oceania."
                "Virtually no EVs were sold in Africa, central Asia, or the middle east."),
                dcc.Graph(id='number-new-EV', figure=jon1)
            ]),
            html.Div([
                html.H2("Replicated Visual 6"),
                html.P("This visualization shows the proportion of ALL car sales that are EVs, between 2010 and 2023. The big takeaway here is that, despite the explosive growth"
                "of EV car sales in recent years, it is still very much a small part of the overall global auto market."),
                dcc.Graph(id='number-new-EV-year', figure=jon2)
            ]),
            html.Div([
                html.H2("Replicated Visual 7"),
                html.P("In this dataset, 'stock' represents the number of cars currently on the road and in use. The plot above shows the share of car stock in each country that is electric."
                "This plot nicely shows the major uptick in EV use over the last few years, with Norway being big early adopters."
                "In 2023 a staggering 1/3 of cars on the road in Norway are electric, with neighbor Sweden in a distant second place with roughly 1/10."
                "Globally only about 3% of cars in active use are currently EVs, but that is a big increase from even 5 years ago."),
                dcc.Graph(id='share-ev-year', figure=jon3)
            ]),
            html.Div([
                html.H2("Replicated Visual 8"),
                html.P("This plot looks similar, but instead of showing a percentage it is showing the total count of EVs on the road in the countries/regions."
                "China is a big country by just about any metric, and they are currently dominating this plot with over 40 Million EVs."
                "This is more than the USA and the EU combined."),
                dcc.Graph(id='stock-ev-year', figure=jon4)
            ])
        ]),

        # Tab 2 - Unique Visuals
        dcc.Tab(label='Unique Visuals', children=[
            html.H2("Unique Visuals", style = {'text-align':'center'}),

            html.Div([
                html.H2("EV Stock Share vs Oil Displacement Among Different Regions"),
                html.P("This graph shows how as EV stock Share increases, so does oil displacement, meaning less consumption of gasoline and diesel fuels."
                " You can see how the world has a pretty steep slope with oil consumption decreasing fast worldwide,"
                " but in countries like China, even with more EV stock share, they still use a lot of oil and fuel."),
                dcc.Graph(id='ev-vs-oil', figure=Dani3)
            ]),
            html.Div([
                html.H2("Electricity Demand and Oil Displacement Over Time"),
                html.P("This graph shows how Electricity Demand is positively correlated to Oil Displacement."
                " This means as Electricity demand goes up, the amount of Oil needed directly goes down as a result."
                " This showcases a shift toward renewable energy sources and a reduction in reliance on fossil fuels."),
                dcc.Graph(id='electricity-vs-oil', figure=Dani4)
            ]),
            html.Div([
                html.H2("EV Stock vs. EV Sales Ratio by Region - 2023"),
                html.P("The ratio of EV stock to EV sales for the most recent year (2023) indicates how many EVs a region has on the road in comparison to how many they are selling."
                "For countries like Norway, this shows that they have a lot of EVS on the road already and the market may already becoming saturated for EVs. In comparison, for countries such as Brazil, India"
                "or Australia that have a lower ratio, it indicates that the country is selling a lot of EVs, but does not have many EVs already on the road as of the most recent year."
                "These countries might demonstrate a desirable or up-and-coming market for EV sales."),
                dcc.Graph(id='EV-Stock-Ratio-Region', figure=mia3)
            ]),
            html.Div([
                html.H2("Proportional EV Sales in USA (2011-2023)"),
                html.P("Annually, EV sales have grown significantly in the US. Proportionally, sales in the 2020's have been significantly higher compared to the 2010's."),
                dcc.Graph(id='Ev-Sales-Proportional', figure=mia4)
            ]),
            html.Div([
                html.H2("EV Sales for BRICS Nations Over Time"),
                html.P("This fun bubble chart shows a comparison of the BRICS nations that were included in this dataset."
                "BRICS officially stands for Brazil, Russia, India, China, and South Africa, but also includes Iran, Egypt, Ethiopia, and the United Arab Emirates."
                "It is an informal group of emerging market economies that are often discussed in business and politics."
                "This plot shows the % of car sales in the BRICS nations, with the size of the circle being a proportion compared to the other countries in the dataset."
                "Again, China dominates here, with nearly 40% of new car sales in 2023, but the UAE is accelerating quickly, and the other countries showing big jumps since 2020."),
                dcc.Graph(id='EV-Sales-BRICS', figure=jon5)
            ]),
            html.Div([
                html.H2("EV Sales by Continent"),
                html.P("This plot splots up the total EV sales into continental boxes, and it backs up the information we saw in the global heatmap."
                "Most EV car sales are happening in Asia, Europe, and North America, with the other continents falling extremely far behind. These may be markets to invest in."),
                dcc.Graph(id='Ev-Sales_continent', figure=jon6)
            ])
        ]),

        # Tab 3 - ML
        dcc.Tab(label='Predictive Models', children=[
            html.H2(" Actual vs. Predicted Graphs", style = {'text-align':'center'}),

            html.Div([
                html.H2("Interactive Model of Predicted vs. Actual"),
                dcc.Graph(id="actual-vs-predicted", figure=sonu1)
            ]),
            html.Div([
                html.H2("Predictive Models Bar Graph"),
                html.P("XGBoost was the best on trained data, but with future data, Linear Regression had the best."),
                dcc.Graph(id='ML-bar', figure=sonu2)
            ]),
            html.Div([
                html.H2("Predictions on Historical Dataset"),
                html.P("Using the Machine Learning models on the data it was trained on."),
                dcc.Graph(id='historical-prediction', figure=sonu3)
            ]),
            html.Div([
                html.H2("Predictive Models by Powertrain"),
                html.P("This visualization shows the predictions of the actual projections vs the machine learning models based on powertrain. Battery electric vehicles (BEV) have the most sales in all models."),
                dcc.Graph(id='powertrain-predictive', figure=sonu4)
            ])
        ]),

        # Tab 4 - Conclusion
        dcc.Tab(label='Conclusion', children=[
            html.H2(" Conclusion ", style = {'text-align':'center'}),
            html.Div([
                html.P("This collection of visualizations show the global trends of electric vehicle (EV) sales globally for the last decade and a half."
                "Some key takeaways are as follows:EV sales are increasing globally, and the rate that they are increasing is also increasing! The world is starting to go electric."
                "Looking at total cars on the road, China is unrivaled. Looking at percentage of cars on the road, or percentage of car sales, the undisputed kings of EVs are the Scandanavian countries of Norway and Sweden."
                "While sales are going up globally, the rate of increase seems to be leveling off in some regions, like Europe."
                "The developing nations categorized as BRICS are lagging behind, but are slowly beginning to catch up in the EV revolution"
                "Full-batter electric cars are much more popular than hybrids in every region, although both are showing growth."
                "As climate change continues to worsen, solutions like EV cars will be critical. The impact that EV cars have on lessening oil demand should not be underestimated. However, our group fears that it may be too-little too-late. Despite the increase in popularity, EV adoption is still very low globally."
                "Perhaps solutions should be sought elsewhere. It is naive to think that the industry that has caused this global crisis is going to be reliably able to also come up with the solution."
                "Out of all the predictive models, XGBoost seems to perform the best with the lowest MSE score, however, all the machine learning models do a poor job of predicting future data."
                "Observing the ML models to actual projected data from the graphs, linear regression seems to perform the best with a steady increase over time."
                "Time series models and neural networks such as SARIMA/ARIMA/LSTM may be better adapted for predicting future trends on this dataset."),
            ])
        ])
    ])
])

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

## Conclusion

In conclusion, this collection of visualizations show the global trends of electric vehicle (EV) sales globally for the last decade and a half.

Some key takeaways are as follows:

*   EV sales are increasing globally, and the rate that they are increasing is also increasing! The world is starting to go electric.
*   Looking at total cars on the road, China is unrivaled.
*   Looking at percentage of cars on the road, or percentage of car sales, the undisputed kings of EVs are the Scandanavian countries of Norway and Sweden.
*   While sales are going up globally, the rate of increase seems to be leveling off in some regions, like Europe.
*   The developing nations categorized as BRICS are lagging behind, but are slowly beginning to catch up in the EV revolution
*   Full-batter electric cars are much more popular than hybrids in every region, although both are showing growth.
*   As climate change continues to worsen, solutions like EV cars will be critical. The impact that EV cars have on lessening oil demand should not be underestimated. However, our group fears that it may be too-little too-late. Despite the increase in popularity, EV adoption is still very low globally.
*   Perhaps solutions should be sought elsewhere. It is naive to think that the industry that has caused this global crisis is going to be reliably able to also come up with the solution.
*   Out of all the predictive models, XGBoost seems to perform the best with the lowest MSE score, however, all the machine learning models do a poor job of predicting future data
*   Observing the ML models to actual projected data from the graphs, linear regression seems to perform the best with a steady increase over time
*   Time series models and neural networks such as SARIMA/ARIMA/LSTM may be better adapted for predicting future trends on this dataset

##Citations

Data Source:
IEA (2024), Global EV Outlook 2024, IEA, Paris https://www.iea.org/reports/global-ev-outlook-2024, License: CC BY 4.0.

International Energy Agency. Global EV Outlook 2024. – processed by Our World in Data. “Electric cars” [dataset]. International Energy Agency. Global EV Outlook 2024., “electric_cars_iea” [original data].